# Supervised Learning with Kernel Methods

Here we use a variational classifier to learn a nonlinear boundary using kernel methods.

The variational circuit rchitecture is specified by [Farhi and Neven (2018)](https://arxiv.org/abs/1802.06002). 
The kernel map is specified by [Havlicek et al (2018)](https://arxiv.org/abs/1804.11326)

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

from scipy.stats import unitary_group

In [2]:
dev = qml.device('default.qubit', wires=2)

In [3]:
def U_phi(x):
    #print(x, np.shape(x))
    # x3 := (pi - x1)(pi - x2)
    x_0, x_1, x_2 = x[0], x[1], x[2]
    #print(x_0, x_1, x_2)
        
    qml.RZ( x_0 , wires=0)
    qml.RZ( x_1 , wires=1)
    
    qml.CNOT(wires=[0,1])
    qml.RZ(x_2,wires=1)
    qml.CNOT(wires=[0,1])

In [4]:
def layer(W): # 6 weights are specified at each layer
    
    # euler angles
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)

    qml.CNOT(wires=[0, 1])

In [5]:
def featuremap(x):
    for i in range(2):
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=1)
        U_phi(x)

In [6]:
@qml.qnode(dev)
def circuit1(weights, x):

    featuremap(x)

    for W in weights:
        layer(W)

    return qml.expval.PauliZ(wires=0)

@qml.qnode(dev)
def circuit2(weights, x):

    featuremap(x)

    for W in weights:
        layer(W)

    return qml.expval.PauliZ(wires=1)

In [7]:
def variational_classifier(var, x): # x is a keyword argument -> fixed (not trained)
    weights = var[0]
    bias = var[1]

    return circuit1(weights, x) * circuit2(weights, x) + bias

In [8]:
def square_loss(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2
    loss = loss / len(labels)

    return loss

In [18]:
def accuracy(labels, predictions):
    print(labels, predictions)
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [10]:
def cost(var, X, Y):

    predictions = [variational_classifier(var, x) for x in X]
    if (len(Y) == num_data):
        print("[(pred, label), ...]: ", list(zip(predictions, Y)))
    return square_loss(Y, predictions) 

In [11]:
random_U = unitary_group.rvs(4)
random_U = random_U / (np.linalg.det(random_U) ** (1/4)) # so that det = 1


print("random unitary: ", random_U)
print("det", np.linalg.det(random_U))

random unitary:  [[-0.18162439-0.0564827 j -0.46921616+0.35182063j -0.18429372+0.71136095j
   0.15362355+0.23723818j]
 [ 0.16715375+0.52652749j  0.45677116+0.07710775j -0.26365744+0.07976064j
  -0.05430152+0.63357499j]
 [ 0.56985238-0.46167089j -0.06118629+0.56898051j -0.03623803-0.27996401j
  -0.07567228+0.22187009j]
 [-0.27638715-0.21068722j -0.22530634-0.25145063j  0.51402037-0.20053536j
  -0.01917148+0.67855121j]]
det (1-1.2836953722228375e-16j)


In [12]:
@qml.qnode(dev)
def data_label_1(x):
    #print(u)
    #print("label the following:", x)
    featuremap(x)
    qml.QubitUnitary(random_U, wires=[0,1])
    
    return qml.expval.PauliZ(wires=0)

@qml.qnode(dev)
def data_label_2(x):
    #print("label the following:", x)
    featuremap(x)
    qml.QubitUnitary(random_U, wires=[0,1])
    
    return qml.expval.PauliZ(wires=1)

In [13]:
thresh = 0.3

X = np.array([])
Y = np.array([])
ctr = 0 # num valid data pts
maxval = 0.0
minval = 0.0

np.random.seed(0)

while ctr < 40:
    x = np.random.rand(2) * 2 * np.pi
    x = np.append(x, (np.pi - x[0]) * (np.pi - x[1]))
    y_1 = data_label_1(x)
    y_2 = data_label_2(x)
    #print(y_1, y_2, y_1 * y_2)
    if (np.abs(y_1 * y_2) > maxval):
        maxval = y_1 * y_2
        print("new max separation: ", maxval)
    elif (y_1 * y_2 < minval):
        minval = y_1 * y_2
        print("new min separation: ", minval)
        
    if y_1 * y_2 > thresh:
        Y = np.append(Y, +1)
        X = np.append(X, x)
        ctr += 1
        print("+1")
    elif y_1 * y_2 < -1 * thresh:
        Y = np.append(Y, -1)
        X = np.append(X, x)
        ctr += 1
        print("-1")

new max separation:  -0.022712201818728222
new max separation:  0.05424512049119921
new max separation:  -0.11149464029858032
new max separation:  0.21272145481532936
new min separation:  -0.02488334739968215
new min separation:  -0.15068202989514268
new max separation:  -0.3994676946123235
-1
new max separation:  -0.006211618700467554
new max separation:  0.1422915430133009
new max separation:  -0.1995703597559318
new max separation:  -0.1220707091424859
new max separation:  0.10902943610614052
new max separation:  0.17724482877543116
new max separation:  -0.26515825998189013
new max separation:  -0.22340171561511674
new max separation:  0.10590256058931216
new max separation:  0.1218657623659504
new max separation:  -0.6053511176490141
-1
new max separation:  0.11505356882003209
new max separation:  0.8540874911941555
+1
new min separation:  -0.2510149117117334
new min separation:  -0.6461257785348852
-1
+1
+1
+1
-1
+1
+1
-1
-1
-1
+1
+1
-1
-1
+1
-1
+1
-1
new min separation:  -0.68408

In [14]:
X = X.reshape(-1, 3)
print(X)
print(Y)

[[ 4.88930306  5.46644755  4.0631731 ]
 [ 1.23516341  2.31676857  1.57246875]
 [ 6.13717092  3.80035648  1.97337861]
 [ 4.61936028  6.04560893  4.29146123]
 [ 5.1132425   2.49131905 -1.28211186]
 [ 5.53613466  3.65224517  1.22277891]
 [ 6.00725065  4.04630976  2.5926098 ]
 [ 4.48771044  6.27594084  4.21920183]
 [ 3.62683419  1.49472468 -0.79912874]
 [ 5.86983967  3.85766187  1.95361371]
 [ 0.15506102  0.4225419   8.12053109]
 [ 6.22248312  1.36280395 -5.48025318]
 [ 1.30357293  2.6683775   0.86977879]
 [ 3.55264688  1.15158117 -0.81800263]
 [ 4.55904385  0.07180084 -4.35128007]
 [ 1.78140145  2.38715147  1.02618426]
 [ 1.63000004  2.34873722  1.19847442]
 [ 5.98080373  3.61755124  1.35134689]
 [ 5.15703191  5.7104335   5.17734269]
 [ 6.21664498  0.41031843 -8.39881113]
 [ 1.4616385   2.18981178  1.59894823]
 [ 5.97501147  1.46662272 -4.74589133]
 [ 1.7117715   2.38168472  1.08653244]
 [ 5.27278076  1.49377595 -3.51180737]
 [ 0.95727285  2.62314426  1.1324571 ]
 [ 6.08083326  3.43617918

In [15]:
num_data = len(Y)
num_train = int(0.5 * num_data)

print(num_data, num_train)

index = np.random.permutation(range(num_data))
X_train = X[index[:num_train]]
Y_train = Y[index[:num_train]]

X_test = X[index[num_train:]]
Y_test = Y[index[num_train:]]

40 20


In [16]:
num_qubits = 2
num_layers = 6
var_init = (0.01 * np.random.randn(num_layers, num_qubits, 3), 0.0)

In [17]:
opt = NesterovMomentumOptimizer(0.01)
batch_size = 5

# train the variational classifier
var = var_init
for it in range(200):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size, ))
    X_train_batch = X_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    var = opt.step(lambda v: cost(v, X_train_batch, Y_train_batch), var)

    # Compute predictions on train and validation set
    predictions_train = [np.sign(variational_classifier(var, x)) for f in X_train]
    predictions_test = [np.sign(variational_classifier(var, x)) for f in X_test]
    
    # Compute accuracy on train and validation set
    acc_train = accuracy(Y_train, predictions_train)
    acc_test = accuracy(Y_test, predictions_test)

    print("Iter: {:5d} | Cost: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f} "
          "".format(it+1, cost(var, X, Y), acc_train, acc_test))

[(pred, label), ...]:  [(0.06633853010549862, -1.0), (0.01373184079976447, -1.0), (-0.10471900942317844, 1.0), (0.00028403747678261773, -1.0), (-0.014165984076751657, 1.0), (-0.05906532345589827, 1.0), (-0.4145266265758054, 1.0), (-0.03660467852163929, -1.0), (-0.01951335843232637, 1.0), (-0.0818233942146746, 1.0), (0.07914809245664634, -1.0), (0.10074695329972971, -1.0), (-0.08051838464324262, -1.0), (-0.15920488967541088, 1.0), (-0.00368363134438833, 1.0), (0.062369524862901426, -1.0), (0.023368584134035204, -1.0), (-0.02700314879217639, 1.0), (0.078334234435845, -1.0), (0.2569602118805494, 1.0), (0.013613109428815528, -1.0), (0.01388880693652942, -1.0), (0.04206677894823284, -1.0), (0.054661388679618644, 1.0), (-0.07722697822167693, -1.0), (-0.3808721306418403, 1.0), (0.11132899080690092, -1.0), (0.001822148261042232, -1.0), (-0.11154177286507022, 1.0), (-0.020228281555113366, -1.0), (0.001588234616479254, 1.0), (0.00616342780524481, -1.0), (-0.01016337382395937, -1.0), (-0.06292352

[(pred, label), ...]:  [(0.11275483172281338, -1.0), (0.10219891737621262, -1.0), (0.09902751871984235, 1.0), (0.0905275509914577, -1.0), (0.10266834399733718, 1.0), (-0.058095845045733646, 1.0), (0.01296421876150354, 1.0), (0.07604450319264489, -1.0), (0.23075473322857054, 1.0), (0.10968732844989582, 1.0), (0.1689292486342269, -1.0), (0.24226620080768888, -1.0), (0.0027648403364185364, -1.0), (0.15414264622803023, 1.0), (0.07224849246610957, 1.0), (0.11465281954694949, -1.0), (0.08421660068870873, -1.0), (0.010634612671276453, 1.0), (0.1756615065841452, -1.0), (0.2681785324713483, 1.0), (0.09492035441191238, -1.0), (0.10722833282568811, -1.0), (0.09719211850842645, -1.0), (0.25768427244115094, 1.0), (0.02883621466788855, -1.0), (-0.3545789950073808, 1.0), (0.16766452941027107, -1.0), (0.10671702696950443, -1.0), (0.24749544059816211, 1.0), (0.04740514884427624, -1.0), (0.10253386592932377, 1.0), (0.0941643223767675, -1.0), (0.027140888792732007, -1.0), (0.09126034069770834, -1.0), (0.

[(pred, label), ...]:  [(0.0740001374959797, -1.0), (0.061821490795226273, -1.0), (0.1858563936038244, 1.0), (0.05000872439213566, -1.0), (0.08182201966331373, 1.0), (-0.1229053016172864, 1.0), (0.30525037848318926, 1.0), (0.041104880992689545, -1.0), (0.33271036341266097, 1.0), (0.17026735486974104, 1.0), (0.144538721770224, -1.0), (0.14804391293913957, -1.0), (-0.07547021917508505, -1.0), (0.3285159123256313, 1.0), (0.04189743404387615, 1.0), (0.08028444005907842, -1.0), (0.04085099886805014, -1.0), (-0.03856360314755619, 1.0), (0.09007784888604682, -1.0), (0.06768549004597685, 1.0), (0.05089369504852192, -1.0), (0.08425067749721392, -1.0), (0.05821769797061507, -1.0), (0.3731179325607689, 1.0), (-0.05642012690431884, -1.0), (-0.39993922361578693, 1.0), (0.14879103596256127, -1.0), (0.08435923122053625, -1.0), (0.43613689828947017, 1.0), (-0.019554887385143216, -1.0), (0.0882738904712093, 1.0), (0.03988584465630541, -1.0), (-0.051758252192692325, -1.0), (0.14038251082684083, -1.0), (

[(pred, label), ...]:  [(-0.19445649899432715, -1.0), (-0.14543541753440759, -1.0), (0.09833158261414122, 1.0), (-0.20662036056907807, -1.0), (-0.15761748041137855, 1.0), (-0.33869176533203776, 1.0), (0.2977833819202687, 1.0), (-0.20895568804444958, -1.0), (0.20152451375457586, 1.0), (0.05409050977814327, 1.0), (-0.037683946720350414, -1.0), (-0.21721136226171406, -1.0), (-0.39872460250191083, -1.0), (0.2503147588078872, 1.0), (-0.1198120188276757, 1.0), (-0.17866929541744495, -1.0), (-0.2043899948917725, -1.0), (-0.21951468910706745, 1.0), (-0.13627459167780862, -1.0), (-0.1655700756244039, 1.0), (-0.15772796585675147, -1.0), (-0.06069529685821286, -1.0), (-0.19782313829606496, -1.0), (0.3035688944280203, 1.0), (-0.33315823781849974, -1.0), (-0.5349843698951556, 1.0), (-0.09999889665641855, -1.0), (-0.09800817914372487, -1.0), (0.3293380011253641, 1.0), (-0.32061571746259754, -1.0), (-0.06501088785816199, 1.0), (-0.18881066507389593, -1.0), (-0.39297952591482493, -1.0), (0.01810244638

[(pred, label), ...]:  [(-0.4565760277256458, -1.0), (-0.25402009195781894, -1.0), (0.1236855475635193, 1.0), (-0.4679362354618974, -1.0), (-0.48487061827064365, 1.0), (-0.4486439514684682, 1.0), (0.3535362435077152, 1.0), (-0.4775491902458772, -1.0), (0.09551374211044644, 1.0), (0.036258496246887734, 1.0), (-0.1493772223970463, -1.0), (-0.637511810821365, -1.0), (-0.661376972195199, -1.0), (0.2237455453941341, 1.0), (-0.17072987970647213, 1.0), (-0.4379523589851587, -1.0), (-0.4258257394152285, -1.0), (-0.272148166011951, 1.0), (-0.2952869747261522, -1.0), (-0.12048852807811258, 1.0), (-0.2967462060763427, -1.0), (-0.12303025950810872, -1.0), (-0.4460029962531674, -1.0), (0.30918886110604915, 1.0), (-0.4721825414122064, -1.0), (-0.45667229739055226, 1.0), (-0.3556400521343623, -1.0), (-0.18701005557972064, -1.0), (0.22239637799304868, 1.0), (-0.5834340248393384, -1.0), (-0.08344173499881465, 1.0), (-0.3024564859217813, -1.0), (-0.685181401890104, -1.0), (-0.009795257532745183, -1.0), 

...

[(pred, label), ...]:  [(-0.32747979814796035, -1.0), (-0.6005076804200029, -1.0), (1.0759004386323094, 1.0), (-0.5476016317679359, -1.0), (0.19512945152590783, 1.0), (0.5362870003666168, 1.0), (0.6478083039876845, 1.0), (-0.3128533001900011, -1.0), (0.6214174436095756, 1.0), (0.9052219386283846, 1.0), (0.2672066532025025, -1.0), (-0.26074108559205345, -1.0), (-0.5048205999315167, -1.0), (0.7926336446284485, 1.0), (0.6310193258659805, 1.0), (-0.3754434474098679, -1.0), (-0.526176773261303, -1.0), (0.9311511512293746, 1.0), (-0.14985935447460014, -1.0), (0.8365656986405501, 1.0), (-0.5058191546617381, -1.0), (-0.2093941277262793, -1.0), (-0.4555095988468204, -1.0), (0.6564168363698529, 1.0), (-0.3751031306503311, -1.0), (0.7276005008066069, 1.0), (-0.14217684740483028, -1.0), (-0.4846681325036739, -1.0), (0.6698126533668184, 1.0), (-0.5841198867854727, -1.0), (0.8962024696948135, 1.0), (-0.5521793465537963, -1.0), (-0.3972161451936863, -1.0), (-0.3594076744335395, -1.0), (-0.47406241245

[(pred, label), ...]:  [(-0.30969383676154566, -1.0), (-0.6430840678163467, -1.0), (1.083385885327104, 1.0), (-0.5616036353651612, -1.0), (0.1839242685904223, 1.0), (0.5428005189652785, 1.0), (0.6539035671961346, 1.0), (-0.33417290923667536, -1.0), (0.6086019271332657, 1.0), (0.9165296562266956, 1.0), (0.24205666906591733, -1.0), (-0.2854673164269665, -1.0), (-0.5094668104386157, -1.0), (0.7916812645867706, 1.0), (0.6074538656126036, 1.0), (-0.36013855987970517, -1.0), (-0.5246791135131346, -1.0), (0.9220375761666502, 1.0), (-0.18128002010022004, -1.0), (0.8224456139607472, 1.0), (-0.527941035023866, -1.0), (-0.25829379672914105, -1.0), (-0.44485720149453983, -1.0), (0.6227164852287104, 1.0), (-0.41901627435187255, -1.0), (0.6973175898229863, 1.0), (-0.12552678529444616, -1.0), (-0.5220297517777399, -1.0), (0.6648434013794345, 1.0), (-0.5844296257672097, -1.0), (0.8374707890415303, 1.0), (-0.600613614035229, -1.0), (-0.37758579587600577, -1.0), (-0.4038761619997137, -1.0), (-0.52353248

[(pred, label), ...]:  [(-0.32507753359256486, -1.0), (-0.6512305681910401, -1.0), (1.0577507340534233, 1.0), (-0.6108920698369946, -1.0), (0.19196008541899137, 1.0), (0.5243726421747842, 1.0), (0.6342494264938564, 1.0), (-0.39429933485682456, -1.0), (0.6136292962747434, 1.0), (0.896379762085018, 1.0), (0.21543066100120717, -1.0), (-0.3367434290535788, -1.0), (-0.5600650379886554, -1.0), (0.794268793327154, 1.0), (0.6081056316732782, 1.0), (-0.37578281387131507, -1.0), (-0.5371476924696571, -1.0), (0.887101981432679, 1.0), (-0.16609437564503057, -1.0), (0.7916414435849103, 1.0), (-0.5231302138387212, -1.0), (-0.301475249050834, -1.0), (-0.4604637846968892, -1.0), (0.6035416901116828, 1.0), (-0.46712067734330665, -1.0), (0.6544357457651917, 1.0), (-0.13739908844791013, -1.0), (-0.5122600893593184, -1.0), (0.6692878724091941, 1.0), (-0.6254847413524122, -1.0), (0.7931801304306895, 1.0), (-0.6268885466626448, -1.0), (-0.4147700145280173, -1.0), (-0.38973671848024816, -1.0), (-0.5168085751

[(pred, label), ...]:  [(-0.2617469113731169, -1.0), (-0.6262950807948504, -1.0), (1.041357631632641, 1.0), (-0.6556714483274586, -1.0), (0.18940944301109236, 1.0), (0.5272030625007501, 1.0), (0.6323667996653237, 1.0), (-0.47607869903400135, -1.0), (0.6481484727281887, 1.0), (0.902576081136584, 1.0), (0.20032110805779005, -1.0), (-0.41107169472782257, -1.0), (-0.5956530625827717, -1.0), (0.8181496094559322, 1.0), (0.6298053858083924, 1.0), (-0.3148576611462466, -1.0), (-0.4791924900104493, -1.0), (0.8508548567637729, 1.0), (-0.12198724641773387, -1.0), (0.7373072300059058, 1.0), (-0.45371398331881707, -1.0), (-0.3793223767131395, -1.0), (-0.40218320875064795, -1.0), (0.6336783097718468, 1.0), (-0.5419786433557572, -1.0), (0.5885029995968234, 1.0), (-0.06992812276629537, -1.0), (-0.4477482364387083, -1.0), (0.7029323149371882, 1.0), (-0.6315212461027816, -1.0), (0.693237818882969, 1.0), (-0.6497591359741443, -1.0), (-0.40108451124836286, -1.0), (-0.3214375286847535, -1.0), (-0.502268629

[(pred, label), ...]:  [(-0.36508202394895656, -1.0), (-0.6774044380406627, -1.0), (0.9832842788897234, 1.0), (-0.6827687260238915, -1.0), (0.12708042139436496, 1.0), (0.43975233921009166, 1.0), (0.5706295934267416, 1.0), (-0.48477337949670024, -1.0), (0.5790112704197349, 1.0), (0.8281613724136543, 1.0), (0.15561325028984424, -1.0), (-0.4623065021259202, -1.0), (-0.6323846253775416, -1.0), (0.7516573016036039, 1.0), (0.5595201891410775, 1.0), (-0.4150004077379092, -1.0), (-0.5716472035706177, -1.0), (0.7915617622792546, 1.0), (-0.20173550399632417, -1.0), (0.7089715351946939, 1.0), (-0.5425427150634587, -1.0), (-0.3760490314523088, -1.0), (-0.4988050635321221, -1.0), (0.5796084804561876, 1.0), (-0.5408922608851341, -1.0), (0.5448588325824952, 1.0), (-0.17403264170693744, -1.0), (-0.5239119604993066, -1.0), (0.6311548335630625, 1.0), (-0.6861427041911072, -1.0), (0.6998772374891089, 1.0), (-0.671508053417714, -1.0), (-0.4788835415597335, -1.0), (-0.3899075020410368, -1.0), (-0.545858141